# Bounding configuration GP example

This notebook is designed to test whether a simple simple comprised of bounding boxes can be generated using [deap](http://deap.readthedocs.io/en/master/api/tools.html), a Python Evolutionary Algorithm Package.

In [1]:
import copy

import numpy as np

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp


In [2]:
# def progn(*args):
#     for arg in args:
#         arg()

# def prog2(out1, out2): 
#     return partial(progn,out1,out2)

# def prog3(out1, out2, out3):     
#     return partial(progn,out1,out2,out3)

# def if_then_else(condition, out1, out2):
#     out1() if condition() else out2()



In [72]:
from OCC.gp import gp_Pnt
from airconics.base import AirconicsShape, AirconicsCollection
import OCC
from airconics import AirCONICStools as act
from OCC.BRepPrimAPI import BRepPrimAPI_MakeBox


# Create a simple Box class that inherits from AirconicsShape
#  - this will behave similarly to the Engine, Fuselage class etc.
class Box(AirconicsShape):
    def __init__(self, xmin, ymin, zmin, dx, dy, dz):
        print("Creating a box!")
        print(xmin, ymin, zmin)
        # This implicitly calls build
        super(Box, self).__init__(construct_geometry=True,
                                             xmin=xmin,
                                             ymin=ymin,
                                             zmin=zmin,
                                             dx=dx,
                                             dy=dy,
                                             dz=dz)
        
        
    def Build(self):
        Xmin = gp_Pnt(self.xmin, self.ymin, self.zmin)
        Xmax = gp_Pnt(self.xmin + self.dx, self.ymin + self.dy, self.zmin + self.dz)
        self['Box'] = BRepPrimAPI_MakeBox(Xmin, Xmax).Shape()

        

# def progn(*args):
#     """LSIP progn connective"""
#     for arg in args:
#         arg()



# def fit2(out1, out2):
#     return partial(fitN, out1, out2)

# def fit2(out1, out2):
#     return partial(fitN, out1, out2)

        
class BoxLayout(AirconicsCollection):
    def __init__(self):
        """maxcompoents is the number of components"""
        super(BoxLayout, self).__init__(parts={}, construct_geometry=True)
        
        print("hello")
        
        # Start with a simple box
        self.nparts = 0
        self.routine = None
    
    def run(self, routine):
        self._reset()
        routine()

    def _reset(self):
        self._Parts = {}
        self.nparts = 0

    def boxN(self, xmin, ymin, zmin, dx, dy, dz, *args):
        # Fits N new components to this box layout
        box = Box(xmin, ymin, zmin, dx, dy, dz)
        self.nparts += 1
        # Do no be confused between the numbering of boxes and the number of descendent
        #  nodes: Each box needs a unique ID, so naming a box0 function "box0" replaces
        #  other shapes in this layout that are also named box0
        self['box' + str(self.nparts)] = box
        for arg in args:
            arg()
    
    def box0(self, xmin, ymin, zmin, dx, dy, dz):
#         box = Box(xmin, ymin, zmin, dx, dy, dz)
#         self.nparts += 1
#         self['box0'] = box
        print("in box0")
        return partial(self.boxN, xmin, ymin, zmin, dx, dy, dz)
        
    # Now we need to explicitly add the box1, box2, .... boxM functions,
    # So that they can be added to the GP toolbox
    def box1(self, xmin, ymin, zmin, dx, dy, dz, out1):
#         box = Box(xmin, ymin, zmin, dx, dy, dz)
#         self.nparts += 1
#         self['box1'] = box
        print("in box1, args: {}, {}, {}, {}, {}, {}, {}".format(xmin, ymin, zmin, dx, dy, dz, out1))
        print("box1 args")
        return partial(self.boxN, xmin, ymin, zmin, dx, dy, dz, out1)

    def box2(self, xmin, ymin, zmin, dx, dy, dz, out1, out2):
#         box = Box(xmin, ymin, zmin, dx, dy, dz)
#         self.nparts += 1
#         self['box2'] = box
        return partial(self.boxN, xmin, ymin, zmin, dx, dy, dz, out1, out2)
    
    def temp_fitness(self):
        """Until I come with something useful, the fitness evaluation for this
        configuration will be based on the bounding box volume"""
        xmin, ymin, zmin, xmax, ymax, zmax = self.Extents()
        return (xmax-xmin) * (ymax-ymin)*(zmax-zmin)


In [99]:
config = BoxLayout()
import types, itertools, functools
funct = types.FunctionType
part = functools.partial
nt = types.NoneType

# I think the second argument means that we have up to 57 floats to use as terminals
pset = gp.PrimitiveSetTyped("MAIN", [], nt)


pset.addPrimitive(config.box0, [float, float, float, float, float, float], nt)
pset.addPrimitive(config.box1, [float, float, float, float, float, float, nt], nt)
pset.addPrimitive(config.box2, [float, float, float, float, float, float, nt, nt], nt)
pset.addPrimitive(np.random.rand, [], float)

# terminals
# for i in range(100):
# label = 'rand' + str(i)
pset.addEphemeralConstant('rand', np.random.rand, float)
# for i in range(100):
#     pset.addTerminal(1.0, float)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("expr_init", gp.genFull, pset=pset, min_=1, max_=2)

# Structure initializers
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr_init)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


# def evalTopology(individual):
#     # Transform the tree expression to functionnal Python code
#     routine = gp.compile(individual, pset)
#     # Run the generated routine
#     global config
#     config.run(routine)
#     return config.temp_fitness,
    
# toolbox.register("evaluate", evalTopology)
# toolbox.register("select", tools.selTournament, tournsize=7)
# toolbox.register("mate", gp.cxOnePoint)
# toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
# toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

# def main():
#     np.random.seed(69)
    
#     pop = toolbox.population(n=100)
#     hof = tools.HallOfFame(1)
#     stats = tools.Statistics(lambda ind: ind.fitness.values)
#     stats.register("avg", np.mean)
#     stats.register("std", np.std)
#     stats.register("min", np.min)
#     stats.register("max", np.max)
    
#     algorithms.eaSimple(pop, toolbox, 0.5, 0.2, 40, stats, halloffame=hof)
    
#     return pop, hof, stats



Attempting to construct BoxLayout geometry...
hello


In [102]:
tree = toolbox.individual()
print(tree)

routine = gp.compile(tree, pset)

config.run(routine)


from airconics.Addons.WebServer.TornadoWeb import TornadoWebRenderer

renderer = TornadoWebRenderer()

config.Display(renderer)

renderer

IndexError: The gp.generate function tried to add a terminal of type '<type 'NoneType'>', but there is none available.

In [84]:
routine = gp.compile(tree, pset)

config.run(routine)
from airconics.Addons.WebServer.TornadoWeb import TornadoWebRenderer

render = TornadoWebRenderer()

config.Display(renderer)

in box0
Creating a box!
(0.21596537503143343, 0.027280711953040826, 0.20580687228223316)


In [7]:
config = BoxLayout(10)

def make_box(xmin, ymin, zmin, dx, dy, dz):
    def box0():
        global config
        config.box0(xmin, ymin, zmin, dx, dy, dz)
        return None
    return box0

p = config.box2(0,0,0,1,1,1, config.box0(1,1,1, 1, 1, 1), config.box0(2,2,2, 1, 1, 1))

res = p()

from airconics.Addons.WebServer.TornadoWeb import TornadoWebRenderer

renderer = TornadoWebRenderer()

print(config)
config.Display(renderer)

renderer

TypeError: __init__() takes exactly 1 argument (2 given)

In [80]:
help(pset.addEphemeralConstant
    )

Help on method addEphemeralConstant in module deap.gp:

addEphemeralConstant(self, name, ephemeral, ret_type) method of deap.gp.PrimitiveSetTyped instance
    Add an ephemeral constant to the set. An ephemeral constant
    is a no argument function that returns a random value. The value
    of the constant is constant for a Tree, but may differ from one
    Tree to another.
    
    :param name: name used to refers to this ephemeral type.
    :param ephemeral: function with no arguments returning a random value.
    :param ret_type: type of the object returned by *ephemeral*.



In [ ]:
help(gp.PrimitiveSetTyped)

In [ ]:
help(gp.PrimitiveSet)

In [ ]:
import itertools

r = [itertools.repeat(float, 57)] + [funct]

for entry in r:
    print(entry)

In [ ]:
from airconics.Addons.WebServer.TornadoWeb import TornadoWebRenderer

renderer = TornadoWebRenderer()
# b = Box(0,0,0,1,1,1)
box = BRepPrimAPI_MakeBox(gp_Pnt(0,0,0), gp_Pnt(1,1,1)).Shape()
renderer.DisplayShape(box)
renderer

In [ ]:
type(program)

In [ ]:
b = program()

In [71]:
np.random.rand()

0.17239200797715737

In [ ]:
from airconics.Addons.WebServer.TornadoWeb import TornadoWebRenderer

renderer = TornadoWebRenderer()
b.Display(renderer)
renderer